In [9]:
import time
import openpyxl
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import copy
import zhconv
from transformers import BertTokenizer, BertModel 
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher,Subgraph

### wikipedia

In [10]:
driver = webdriver.Chrome()
url = 'https://zh.wikipedia.org/wiki/Category:%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD'
driver.get(url)
time.sleep(2)

Hierarchy_list=[]
wait_list=[]
wait_list.append(['https://zh.wikipedia.org/wiki/Category:%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD','人工智能'])

In [21]:
visited=[]
def get_Hierarchy(wait_list):
    if len(wait_list)==0:
        return

    wait_list_=[]
    for w_i in wait_list:
        father = w_i[1]
        visited.append(father)
        url = w_i[0]
        driver.get(url)
        
        sub=driver.find_elements_by_xpath('//*[@id="mw-subcategories"]/div/div/div/ul/li/div/div/a')
        for s_i in sub:
            son = s_i.text
            if son in Visited:
                continue
            else:
                wait_list_.append([s_i.get_attribute('href'),s_i.text])
                Hierarchy_list.append([father,son])
                
        time.sleep(5)
        
    get_Hierarchy(wait_list_)

In [26]:
get_Hierarchy(wait_list)

In [27]:
# 繁体处理
name=['father','son']
Hierarchy_df=pd.DataFrame(columns=name,data=Hierarchy_list)


for i in range(len(hy_Hierarchy)):
    Hierarchy_df.iloc[i,0] = zhconv.convert(Hierarchy_df.iloc[i,0], 'zh-cn')
    Hierarchy_df.iloc[i,1] = zhconv.convert(Hierarchy_df.iloc[i,1], 'zh-cn')  
    
Hierarchy_df.to_csv('./DATA/Hierarchical.csv',encoding="utf_8_sig")

### Ontology_Problems

In [28]:
Hierarchy_df = pd.read_csv('./DATA/Hierarchical.csv',index_col=0).reset_index(drop=True)

In [29]:
problem = pd.read_csv('./DATA/problems.csv')

In [30]:
pretrained_weights="../bert-base/bert-base-chinese/"
model = BertModel.from_pretrained(pretrained_weights)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

In [31]:
def corr(sentence_0,sentence_1):
    input_ids_0 = torch.tensor(tokenizer.encode(sentence_0)).unsqueeze(0)
    outputs_0 = model(input_ids_0)
    word_o = outputs_0[0][0][1:len(sentence_0)+1].mean(axis=1).detach().numpy()
#     print(word_o)
    
    input_ids_0 = torch.tensor(tokenizer.encode(sentence_1)).unsqueeze(0)
    outputs_1 = model(input_ids_0)
    word_p = outputs_1[0][0][1:len(sentence_1)+1].mean(axis=1).detach().numpy()
    
    cos_sim1 = word_p.dot(word_o) / (np.linalg.norm(word_p) * np.linalg.norm(word_o))
    
    return cos_sim1

In [32]:
connected=[[]]

In [33]:
for p in problem:
    cos=[]
    input_p = torch.tensor(tokenizer.encode(p)).unsqueeze(0)
    outputs_p = model(input_p)
    word_p = outputs_p[0][0][1:len(p)+1].sum(axis=0).detach().numpy()
    
    for o in Ontology_text:
        input_o = torch.tensor(tokenizer.encode(o)).unsqueeze(0)
        outputs_o = model(input_o)
        word_o = outputs_o[0][0][1:len(o)+1].sum(axis=0).detach().numpy()
        
        cos_sim1 = word_p.dot(word_o) / (np.linalg.norm(word_p) * np.linalg.norm(word_o))
        cos.append(cos_sim1)
    
    max_value=max(cos)
    max_index = [i for i,j in enumerate(cos) if j==max_value]
    
    connected.append([p,np.array(Ontology_text)[max_index].tolist()])

In [34]:
on = pd.DataFrame(data=connected)
con.to_csv('./DATA/connected.csv',encoding='utf_8_sig')